In [160]:
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup

In [161]:
page_count = 0

def custom_extractor(html):
    global page_count
    page_count += 1
    print(f"Page count: {page_count}")
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

chaiDocsLoder = RecursiveUrlLoader(
    url="https://docs.chaicode.com/",
    # url="https://python.langchain.com/",
    max_depth=500,
    timeout=60,
    extractor=lambda html: BeautifulSoup(html, "html.parser").text
    # extractor=custom_extractor
)


In [162]:
docs = chaiDocsLoder.load()
print(f"Number of documents loaded: {len(docs)}")

Number of documents loaded: 83


In [163]:
docs[0].page_content

' Home | Chai aur Docs\n  Skip to content        Chai aur Docs        YouTube Instagram LinkedIn GitHub X                                          Docs You’ll  Actually Read Next-gen docs that builds reading habits into your workflow.     Start Learning     Watch Tutorials       \n Have a question or want to get involved..?    Join our Discord     \n  Brought to you by ChaiCode            Chai aur Code is an unique initiative by Hitesh Choudhary where he mentors people who want to learn programming and grow in the field.Learn about ChaiCode         \nContribute\n    \nCommunity\n   \nSponsor\n          '

In [164]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20, )
chaiCodedocs = text_splitter.split_documents(docs)

In [165]:
for doc in chaiCodedocs:
    print(doc.page_content)


Home | Chai aur Docs
Skip to content        Chai aur Docs        YouTube Instagram LinkedIn GitHub X                                          Docs You’ll  Actually Read Next-gen docs that builds reading habits into
reading habits into your workflow.     Start Learning     Watch Tutorials
Have a question or want to get involved..?    Join our Discord
Brought to you by ChaiCode            Chai aur Code is an unique initiative by Hitesh Choudhary where he mentors people who want to learn programming and grow in the field.Learn about ChaiCode
about ChaiCode
Contribute
    
Community
   
Sponsor
Getting Started | Chai aur Docs
Skip to content        Chai aur Docs        Search  CtrlK      Cancel             YouTube Instagram LinkedIn GitHub X                                            Getting Started       Chai aur HTML
Chai aur HTML       Welcome    HTML Intro    Emmet Crash Course    Common HTML Tags          Chai aur Git       Welcome    Git and GitHub    Terminology    Behind the scenes

In [166]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
Api_key = os.getenv("GOOGLE_API_KEY")
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/text-embedding-004",
    api_key = Api_key,
    dimesions=1024
)

In [167]:
from langchain_community.vectorstores import FAISS
ChaiCodeVectordb = FAISS.from_documents(chaiCodedocs, embeddings)
ChaiCodeVectordb.save_local("ChaiCodeVectordb")

In [170]:
newVectorDB = FAISS.load_local("ChaiCodeVectordb", embeddings,allow_dangerous_deserialization=True)
query = "kya tum mujhe loops in C++ ke baare me full detals me bta sakte ho with example for every loop and code also"
response = newVectorDB.similarity_search(query)
response # to get the source of the document

[Document(id='093d89d4-6c60-464b-b8c9-9c42dfa0e761', metadata={'source': 'https://docs.chaicode.com/youtube/chai-aur-c/loops/', 'content_type': 'text/html; charset=UTF-8', 'title': 'Loops | Chai aur Docs', 'description': 'Learn about loops in C++.', 'language': 'en'}, page_content='Summary\nIn this chapter, we have learned about loops in C++. We have also learned about the different types of loops and how to use them.\nStart your journey with ChaiCode'),
 Document(id='c22867a4-c70a-4fd2-85ef-c34179ca014d', metadata={'source': 'https://docs.chaicode.com/youtube/chai-aur-c/loops/', 'content_type': 'text/html; charset=UTF-8', 'title': 'Loops | Chai aur Docs', 'description': 'Learn about loops in C++.', 'language': 'en'}, page_content='Solution    Going through the code:      Summary           Loops      In this chapter, we will learn about loops in C++. Our learning will be based on the task based approach. Each task will help'),
 Document(id='623561c8-acba-40cf-a17b-3170679fc8e9', metada

In [171]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


from langchain_google_genai import ChatGoogleGenerativeAI

llm= ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.8,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
)

In [172]:
sources = list({doc.metadata["source"] for doc in response})
print(sources)
prompt= ChatPromptTemplate.from_template(
    '''
    You are a helpful assistant. Answer the question based on the context provided below.
    Context: {context} 
    Question: {input}
    Answer:
    '''
)

document_chain = create_stuff_documents_chain(llm,prompt)

['https://docs.chaicode.com/youtube/chai-aur-c/loops/']


In [173]:
result = document_chain.invoke(
    {
        "input": query,
        "context": response # this is a document object not a string
    }
)
final_answer = f"{result}\n\nSource: ({sources})"
print(final_answer)
# print("answer is: ",result)

Context ke anusaar, main aapko C++ mein loops ke baare mein full details mein bata sakta hoon, har loop ke liye example aur code ke saath:

**Loops in C++**

Loops ka upyog code ke ek block ko baar-baar execute karne ke liye kiya jata hai, jab tak ki ek nirdharit shart poori na ho jaye. C++ mein teen prakar ke loop hote hain:

1.  **For Loop:** For loop ek nirdharit sankhya mein ek code block ko execute karne ke liye upyog kiya jata hai.

    *   **Syntax:**

    ```c++
    for (initialization; condition; increment/decrement) {
      // code to be executed
    }
    ```

    *   **Example:**

    ```c++
    #include <iostream>
    
    int main() {
      for (int i = 0; i < 5; i++) {
        std::cout << "Value of i: " << i << std::endl;
      }
      return 0;
    }
    ```

    *   **Output:**

    ```
    Value of i: 0
    Value of i: 1
    Value of i: 2
    Value of i: 3
    Value of i: 4
    ```

2.  **While Loop:** While loop ek code block ko execute karta hai jab tak ki ek nirdh